In [1]:
import sys
sys.path.append('../code')

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import cPickle as pickle
import json

In [ ]:
from make_patch_df import 

In [26]:
# example of intended df style
changes = pd.DataFrame(columns=('Hero', 'Text', 'num_buffs', 'num_nerfs'))
rows_list = []
rows_list.append({'hero_name': 'Abaddon', 'patch_text': ["Mist Coil cooldown reduced from 5 to 4.5", "Borrowed Time Aghanim's Scepter damage redirection increased from 35% to 50%"], 'num_buffs': 2, 'num_nerfs': 0})
rows_list.append({'hero_name': 'Alchemist', 'patch_text': ["Acid Spray damage increased from 12/16/20/24 to 15/20/25/30", "Acid Spray damage increased from 12/16/20/24 to 15/20/25/30"], 'num_buffs': 2, 'num_nerfs': 0})
pd.DataFrame(rows_list)

,hero_name,num_buffs,num_nerfs,patch_text
0,Abaddon,2,0,"[Mist Coil cooldown reduced from 5 to 4.5, Bor..."
1,Alchemist,2,0,[Acid Spray damage increased from 12/16/20/24 ...


In [3]:
# SCRAPE FOR HERO NAME SET

r_heroes = requests.get('http://www.dota2.com/heroes/')
soup_heroes = BeautifulSoup(r_heroes.content, 'html.parser')
hero_name_set = set()
for column in soup_heroes.findAll('div', {'class': 'heroIcons'}):
    for a_tag in column.findAll('a'):
        hero_name_set.add(a_tag['href'].split('/')[-2].replace('_', ' '))

hero_name_set.remove('Natures Prophet')
hero_name_set.add("Nature's Prophet") # dota2.com doesn't include apostrophe

In [4]:
def build_df(hero_change_dict, hero_name_set, patch_name):
    '''
    helper function to assemble dictionary for df_from_patch_query
    '''
    heroes = []
    change_text = []
    for hero, text_list in hero_change_dict.iteritems():
        heroes.append(hero)
        change_text.append(text_list)
        
    df = pd.DataFrame({'hero': heroes, 'text': change_text, 'patch': [patch_name]*len(heroes)})
    changed_heroes_685 = set(df.hero.values)
    untouched_heroes = []
    for hero in hero_name_set:
        if not hero in changed_heroes_685:
            untouched_heroes.append(hero)
    
    df_all_heroes = pd.concat([df, pd.DataFrame({'hero': untouched_heroes,
                                                 'text': [[]]*len(untouched_heroes),
                                                 'patch': [patch_name]*len(untouched_heroes)})])    
    df_all_heroes.sort('hero', inplace=True)
    df_all_heroes.reset_index(inplace=True)
    df_all_heroes.drop('index', axis=1, inplace=True)

    return df_all_heroes

In [5]:
def df_from_patch_query(page, hero_name_set, patch_name):
    '''
    INPUT: patch page url as string, set of acceptable hero names, patch label as string
    OUTPUT: dataframe of hero : text of changes : patch label
    '''
    r = requests.get('https://dota2.gamepedia.com/{0}'.format(page))
    soup = BeautifulSoup(r.content, 'html.parser')

    hero_change_dict = defaultdict(list)
    for ul in soup.find('div', {'id': 'mw-content-text'}).find('div', recursive=False).findAll('ul', recursive=False):
        try:
            li = ul.find('li')
            if li.find('a')['title'] in hero_name_set:
                for sub_ul in li.findAll('ul', recursive=False):
                    for sub_li in sub_ul.findAll('li', recursive=False):
                        hero_change_dict[li.find('a')['title']].append(sub_li.text.strip())
        except:
            continue
    
    df = build_df(hero_change_dict, hero_name_set, patch_name)
    
    return df

In [6]:
df = df_from_patch_query('September_24,_2015_Patch', hero_name_set, '6.85')

In [7]:
df.loc[3]

hero                                             Anti-Mage
patch                                                 6.85
text     [Blink distance rescaled from 1000/1075/1150/1...
Name: 3, dtype: object

In [8]:
# USED FOR MANUAL DATA ENTRY
# df_all.to_csv('../data/hero_table_685.csv')
df_xl = pd.read_csv('../data/hero_table_685_xl.csv')

In [9]:
df_changes = pd.concat([df, df_xl[['num_buffs', 'num_nerfs', 'change_type']]], axis=1)

In [10]:
df_changes['change_type'] = df_changes['change_type'].apply(lambda x: x.replace('[', '').replace(']', '').split(','))
df_changes['change_type'] = df_changes['change_type'].apply(lambda x: [int(num.strip()) for num in x] if len(x[0]) > 0 else [])

In [11]:
df_changes.head(6)

,hero,patch,text,num_buffs,num_nerfs,change_type
0,Abaddon,6.85,"[Mist Coil cooldown reduced from 5 to 4.5, Bor...",2,0,"[1, 1]"
1,Alchemist,6.85,[Acid Spray damage increased from 12/16/20/24 ...,2,0,"[1, 1]"
2,Ancient Apparition,6.85,[Cold Feet cast range increased from 700 to 70...,1,0,[1]
3,Anti-Mage,6.85,[Blink distance rescaled from 1000/1075/1150/1...,0,2,"[-1, -1]"
4,Arc Warden,6.85,[Spark Wraith cooldown reduced from 7 to 4],1,0,[1]
5,Axe,6.85,[],0,0,[]


In [12]:
# Oops! Pudge and Shadow Shaman have an additional fix listed on wiki.
df_changes['change_type'][69] = [1,1,1]
df_changes['change_type'][78] = [1,1,1]

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [13]:
with open('../../dota2-database/json/abilities.json') as f:
    abi_json = json.load(f)
ability_set = set(ability['localizedName'] for ability in abi_json)
ability_set.remove('')

In [17]:
def remove_abi_names(changelist, abi_set=None):
    changes_trimmed = []
    for i, item in enumerate(changelist):
        changes_trimmed.append(item)
        for abi in abi_set:
            item = item.replace(abi, '').strip()
            changes_trimmed[i] = item
    return changes_trimmed

In [19]:
df_changes['text_no_abi'] = df_changes['text'].apply(remove_abi_names, abi_set=ability_set)

In [20]:
df_changes.head(3)

,hero,patch,text,num_buffs,num_nerfs,change_type,text_no_abi
0,Abaddon,6.85,"[Mist Coil cooldown reduced from 5 to 4.5, Bor...",2,0,"[1, 1]","[cooldown reduced from 5 to 4.5, Aghanim's Sce..."
1,Alchemist,6.85,[Acid Spray damage increased from 12/16/20/24 ...,2,0,"[1, 1]",[damage increased from 12/16/20/24 to 15/20/25...
2,Ancient Apparition,6.85,[Cold Feet cast range increased from 700 to 70...,1,0,[1],[cast range increased from 700 to 700/800/900/...


In [123]:
with open('../data/patch_685_df.pkl', 'w') as f:
    pickle.dump(df_changes, f)